# Air Quality Index Data Gathering & Analysis


In [1]:
# Import necessary packages

import pandas as pd

### AQI Data Gathering

In [5]:
# Read in the Air Quality Index CSV files to a dataframe

aqi_df = pd.DataFrame()

folder_dir = "AQI Data"

for year in range(2000, 2024, 1):
    file = folder_dir + "\\" + "daily_aqi_by_cbsa_" + str(year) + ".csv"
    df = pd.read_csv(file)
    aqi_df = pd.concat([aqi_df, df], ignore_index = True)
    
    
# Create dataframes by Defining Parameter - narrowing focus to PM 2.5 and CO
    
aqi_pm25_df = aqi_df[aqi_df["Defining Parameter"] == "PM2.5"]
aqi_co_df = aqi_df[aqi_df["Defining Parameter"] == "CO"]


# Concatenate dataframes to create a filtered AQI dataset

aqi_filtered_df = pd.concat([aqi_pm25_df, aqi_co_df], ignore_index = True)


# Display first 5 rows and header of dataset 

aqi_filtered_df.head()

,CBSA,CBSA Code,Date,AQI,Category,Defining Parameter,Defining Site,Number of Sites Reporting
0,"Aberdeen, SD",10100,2000-02-15,56,Moderate,PM2.5,46-013-0003,1
1,"Aberdeen, SD",10100,2000-02-18,68,Moderate,PM2.5,46-013-0003,1
2,"Aberdeen, SD",10100,2000-02-21,77,Moderate,PM2.5,46-013-0003,1
3,"Aberdeen, SD",10100,2000-02-24,58,Moderate,PM2.5,46-013-0003,1
4,"Aberdeen, SD",10100,2000-02-27,19,Good,PM2.5,46-013-0003,1


### AQI Data Cleaning

In [7]:
# Standardizing "CBSA" format: "First City, State Grouping" - keeping only the first city in a grouping 

aqi_filtered_df[["City", "State"]] = aqi_filtered_df["CBSA"].str.split(",", expand = True)

aqi_filtered_df["City 2"] = aqi_filtered_df["City"].str.extract(r'(^\w+\s*\w*)', expand = False)

aqi_filtered_df["CBSA New"] = aqi_filtered_df["City 2"] + "," + aqi_filtered_df["State"]


# Clean up the extra columns created during cleaning - keep only the "CBSA New" column and rename it "CBSA"

aqi_filtered_df = aqi_filtered_df.drop(["CBSA", "City", "State", "City 2"], axis = 1)

aqi_filtered_df = aqi_filtered_df.rename(columns = {"CBSA New" : "CBSA"})


# Display the first 5 rows and header of the new dataframe

aqi_filtered_df.head()                   

,CBSA Code,Date,AQI,Category,Defining Parameter,Defining Site,Number of Sites Reporting,CBSA
0,10100,2000-02-15,56,Moderate,PM2.5,46-013-0003,1,"Aberdeen, SD"
1,10100,2000-02-18,68,Moderate,PM2.5,46-013-0003,1,"Aberdeen, SD"
2,10100,2000-02-21,77,Moderate,PM2.5,46-013-0003,1,"Aberdeen, SD"
3,10100,2000-02-24,58,Moderate,PM2.5,46-013-0003,1,"Aberdeen, SD"
4,10100,2000-02-27,19,Good,PM2.5,46-013-0003,1,"Aberdeen, SD"


### CBSA Coordinates Data Gathering

In [9]:
# Read in Core-Based Statitical Area (CBSA) Coordinates text file into dataframe

cbsa_df = pd.read_table("2023_Gaz_cbsa_national.txt")


# Display top 5 rows and header of dataframe 

cbsa_df.head()

,CSAFP,GEOID,NAME,CBSA_TYPE,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
0,101.0,10180,"Abilene, TX Metro Area",1,7105668291,36756436,2743.514,14.192,32.452022,-99.718743
1,220.0,10300,"Adrian, MI Micro Area",2,1941553343,30470381,749.638,11.765,41.896022,-84.074356
2,364.0,10380,"Aguadilla, PR Metro Area",1,769374622,354393992,297.057,136.832,18.391555,-67.097202
3,184.0,10420,"Akron, OH Metro Area",1,2331510427,62132699,900.201,23.990,41.146639,-81.350110
4,440.0,10540,"Albany, OR Metro Area",1,5929797336,50130716,2289.508,19.356,44.488898,-122.537208


### CBSA Coordinates Data Cleaning

In [10]:
# Clean up column headers (found due to problem merging datasets later in notebook) 

cbsa_df = cbsa_df.rename(columns = {"INTPTLONG                                                                                                             " : "INTPTLONG"})

cbsa_df.columns

Index(['CSAFP', 'GEOID', 'NAME', 'CBSA_TYPE', 'ALAND', 'AWATER', 'ALAND_SQMI',
       'AWATER_SQMI', 'INTPTLAT', 'INTPTLONG'],
      dtype='object')

In [11]:
# Standardizing "CBSA" format: "First City, State Grouping" - keeping only the first city in a grouping 

cbsa_df[["City", "State"]] = cbsa_df["NAME"].str.split(",", expand = True)

cbsa_df["State Truncated"] = cbsa_df["State"].str.slice(stop = -11)

cbsa_df["City 2"] = cbsa_df["City"].str.extract(r'(^\w+\s*\w*)', expand = False)

cbsa_df["CBSA New"] = cbsa_df["City 2"] + "," + cbsa_df["State Truncated"]


# Clean up the extra columns created during cleaning - keep only the "CBSA New" column and rename it "CBSA"

cbsa_df = cbsa_df.drop( ["NAME", "City", "State", "State Truncated", "City 2"], axis = 1)

cbsa_df = cbsa_df.rename(columns = {"CBSA New": "CBSA"})


# Display the first 5 rows and header of the new dataframe

cbsa_df.head()

,CSAFP,GEOID,CBSA_TYPE,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG,CBSA
0,101.0,10180,1,7105668291,36756436,2743.514,14.192,32.452022,-99.718743,"Abilene, TX"
1,220.0,10300,2,1941553343,30470381,749.638,11.765,41.896022,-84.074356,"Adrian, MI"
2,364.0,10380,1,769374622,354393992,297.057,136.832,18.391555,-67.097202,"Aguadilla, PR"
3,184.0,10420,1,2331510427,62132699,900.201,23.990,41.146639,-81.350110,"Akron, OH"
4,440.0,10540,1,5929797336,50130716,2289.508,19.356,44.488898,-122.537208,"Albany, OR"


In [12]:
# Troubleshooting different naming conventions among CBSAs in the two datasets

cbsa_df["CBSA"] = cbsa_df["CBSA"].str.replace("Lebanon, NH-VT", "Claremont, NH-VT")
cbsa_df["CBSA"] = cbsa_df["CBSA"].str.replace("Española, NM", "Espa, NM")
cbsa_df["CBSA"] = cbsa_df["CBSA"].str.replace("Joplin, MO-KS", "Joplin, MO")
cbsa_df["CBSA"] = cbsa_df["CBSA"].str.replace("Gardnerville Ranchos, NV-CA", "Gardnerville Ranchos, NV")
cbsa_df["CBSA"] = cbsa_df["CBSA"].str.replace("Mayagüez", "Mayaguez")
cbsa_df["CBSA"] = cbsa_df["CBSA"].str.replace("Massena, NY", "Ogdensburg, NY")
cbsa_df["CBSA"] = cbsa_df["CBSA"].str.replace("Prescott Valley", "Prescott")
cbsa_df["CBSA"] = cbsa_df["CBSA"].str.replace("Sioux Falls, SD-MN", "Sioux Falls, SD")
cbsa_df["CBSA"] = cbsa_df["CBSA"].str.replace("Evanston, WY-UT", "Evanston, WY")

aqi_filtered_df["CBSA"] = aqi_filtered_df["CBSA"].str.replace("Chicago, IL-IN-WI", "Chicago, IL-IN")
aqi_filtered_df["CBSA"] = aqi_filtered_df["CBSA"].str.replace("Evansville, IN-KY", "Evansville, IN")
aqi_filtered_df["CBSA"] = aqi_filtered_df["CBSA"].str.replace("Fayetteville, AR-MO", "Fayetteville, AR")
aqi_filtered_df["CBSA"] = aqi_filtered_df["CBSA"].str.replace("Myrtle Beach, SC-NC", "Myrtle Beach, SC")
aqi_filtered_df["CBSA"] = aqi_filtered_df["CBSA"].str.replace("New York, NY-NJ-PA", "New York, NY-NJ")
aqi_filtered_df["CBSA"] = aqi_filtered_df["CBSA"].str.replace("Salisbury, MD-DE", "Salisbury, MD")
aqi_filtered_df["CBSA"] = aqi_filtered_df["CBSA"].str.replace("Worcester, MA-CT", "Worcester, MA")
aqi_filtered_df["CBSA"] = aqi_filtered_df["CBSA"].str.replace("Youngstown, OH-PA", "Youngstown, OH")
aqi_filtered_df["CBSA"] = aqi_filtered_df["CBSA"].str.replace("Fort Madison, IA-IL-MO", "Fort Madison, IA")
aqi_filtered_df["CBSA"] = aqi_filtered_df["CBSA"].str.replace("Greenfield Town", "Greenfield")

### Merging AQI and CBSA Coordinate Datasets

In [13]:
# Merging the CBSA Coordinates dataset together with the AQI dataset on the "CBSA" columns so that each AQI datapoint has a specific location on the map

aqi_cbsa_df = aqi_filtered_df.merge(cbsa_df, on = "CBSA", how = "outer")


# Display the top 5 rows and header of the merged dataframe

aqi_cbsa_df.head()

,CBSA Code,Date,AQI,Category,Defining Parameter,Defining Site,Number of Sites Reporting,CBSA,CSAFP,GEOID,CBSA_TYPE,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
0,10100.0,2000-02-15,56.0,Moderate,PM2.5,46-013-0003,1.0,"Aberdeen, SD",NaN,10100.0,2.0,7.353004e+09,111384253.0,2839.011,43.006,45.519435,-98.7007
1,10100.0,2000-02-18,68.0,Moderate,PM2.5,46-013-0003,1.0,"Aberdeen, SD",NaN,10100.0,2.0,7.353004e+09,111384253.0,2839.011,43.006,45.519435,-98.7007
2,10100.0,2000-02-21,77.0,Moderate,PM2.5,46-013-0003,1.0,"Aberdeen, SD",NaN,10100.0,2.0,7.353004e+09,111384253.0,2839.011,43.006,45.519435,-98.7007
3,10100.0,2000-02-24,58.0,Moderate,PM2.5,46-013-0003,1.0,"Aberdeen, SD",NaN,10100.0,2.0,7.353004e+09,111384253.0,2839.011,43.006,45.519435,-98.7007
4,10100.0,2000-02-27,19.0,Good,PM2.5,46-013-0003,1.0,"Aberdeen, SD",NaN,10100.0,2.0,7.353004e+09,111384253.0,2839.011,43.006,45.519435,-98.7007


In [14]:
# Input missing CBSA coordinates into dataframe 

missing_coordinates = ['Berlin, NH-VT', 'Cullowhee, NC', 'Georgetown, SC',
       'Glenwood Springs, CO', 'Helena, AR', 'Walterboro, SC',
       'Marshall, TX', 'London, KY', 'Point Pleasant, WV-OH',
       'Adjuntas, PR', 'Prineville, OR', 'Madera, CA', 'Havre, MT',
       'New Castle, PA', 'Oxford, NC', 'Brownsville, TN']

latitudes = [44.4687, 35.3137, 33.3768, 39.5505, 34.5292, 32.9052, 32.5449, 37.1290, 38.8445, 18.1967, 44.2998, 36.9613, 48.5500, 41.0037, 36.3107, 35.5940 ]

longitudes = [-71.1851, -83.1765, -79.2945, -107.3248, -90.5912, -80.6668, -94.3674, -84.0833, -82.1371, -66.7367, -120.8345, -120.0607, -109.6841, -80.3470, -78.5908, -89.2623 ]

for i in range(len(missing_coordinates)):
    aqi_cbsa_df.loc[aqi_cbsa_df["CBSA"] == missing_coordinates[i], "INTPTLAT"] = latitudes[i]
    aqi_cbsa_df.loc[aqi_cbsa_df["CBSA"] == missing_coordinates[i], "INTPTLONG"] = longitudes[i]
    
    

In [15]:
# Create dataframes that hold missing coordinates and print them out to check our work
# Empty dataframes means we cleaned our datasets well and we can move on

nan_values_lat = aqi_cbsa_df[aqi_cbsa_df["INTPTLAT"].isna()]
nan_values_long = aqi_cbsa_df[aqi_cbsa_df["INTPTLONG"].isna()]

print(nan_values_lat)
print(nan_values_long)

print(nan_values_lat["CBSA"].unique())
print(nan_values_long["CBSA"].unique())

Empty DataFrame
Columns: [CBSA Code, Date, AQI, Category, Defining Parameter, Defining Site, Number of Sites Reporting, CBSA, CSAFP, GEOID, CBSA_TYPE, ALAND, AWATER, ALAND_SQMI, AWATER_SQMI, INTPTLAT, INTPTLONG]
Index: []
Empty DataFrame
Columns: [CBSA Code, Date, AQI, Category, Defining Parameter, Defining Site, Number of Sites Reporting, CBSA, CSAFP, GEOID, CBSA_TYPE, ALAND, AWATER, ALAND_SQMI, AWATER_SQMI, INTPTLAT, INTPTLONG]
Index: []
[]
[]


In [16]:
# Narrow down merged AQI & Coordinates dataframe to just datapoints within our designated "East Coast" range

east_range = [-87.75, 25.0, -52.75, 62.5]

aqi_cbsa_east_df = aqi_cbsa_df[(aqi_cbsa_df["INTPTLONG"] >= east_range[0]) & (aqi_cbsa_df["INTPTLAT"] >= east_range[1]) & (aqi_cbsa_df["INTPTLONG"] <= east_range[2]) & (aqi_cbsa_df["INTPTLAT"] <= east_range[3])].copy()


# Display top 5 rows and header of east coast dataframe

aqi_cbsa_east_df.head()

,CBSA Code,Date,AQI,Category,Defining Parameter,Defining Site,Number of Sites Reporting,CBSA,CSAFP,GEOID,CBSA_TYPE,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
3279,10420.0,2000-01-01,82.0,Moderate,PM2.5,39-153-0017,3.0,"Akron, OH",184.0,10420.0,1.0,2.331510e+09,62132699.0,900.201,23.99,41.146639,-81.35011
3280,10420.0,2000-01-02,64.0,Moderate,PM2.5,39-153-0017,3.0,"Akron, OH",184.0,10420.0,1.0,2.331510e+09,62132699.0,900.201,23.99,41.146639,-81.35011
3281,10420.0,2000-01-03,52.0,Moderate,PM2.5,39-153-0017,3.0,"Akron, OH",184.0,10420.0,1.0,2.331510e+09,62132699.0,900.201,23.99,41.146639,-81.35011
3282,10420.0,2000-01-04,47.0,Good,PM2.5,39-153-0017,4.0,"Akron, OH",184.0,10420.0,1.0,2.331510e+09,62132699.0,900.201,23.99,41.146639,-81.35011
3283,10420.0,2000-01-06,52.0,Moderate,PM2.5,39-153-0017,3.0,"Akron, OH",184.0,10420.0,1.0,2.331510e+09,62132699.0,900.201,23.99,41.146639,-81.35011


### Export Final Dataset to Parquet

In [17]:
aqi_cbsa_east_df.to_parquet("Air Quality Database.parquet")